# Старостенок Дмитрий КЭ-403 - Классификация с помощью дерева решений

1. Разработайте программу, которая выполняет классификацию заданного набора данных с помощью дерева решений. Параметрами программы являются набор данных, критерий выбора атрибута разбиения (Information gain, Gain ratio, Gini index).

2. Проведите эксперименты на наборе Census Income (данные о результатах переписи населения, в т.ч. о годовом доходе -- ниже или выше $50000: скачать обучающую выборку в формате CSV, тестовую выборку в формате CSV, скачать описание). В качестве обучающей выборки для построения дерева используйте 100% исходных данных.

3. Выполните визуализацию построенных деревьев решений.

4. Доработайте программу, добавив в список ее параметров долю, которую занимает обучающая выборка от общего размера набора данных, и обеспечив вычисление и выдачу в качестве результатов следующих показателей качества классификации: аккуратность (accuracy), точность (precision), полнота (recall), F-мера.
Проведите эксперименты на наборе данных, фиксируя критерий выбора атрибута разбиения и варьируя соотношение мощностей обучающей и тестовой выборок от 60%:40% до 90%:10% с шагом 10%.

5. Выполните визуализацию полученных результатов в виде следующих диаграмм:
    - построенные деревья решений для заданного набора данных;
    - показатели качества классификации в зависимости от соотношения мощностей обучающей и тестовой выборок для заданного набора данных.

6. Подготовьте отчет о выполнении задания и загрузите отчет в формате PDF в систему. Отчет должен представлять собой связный и структурированный документ со следующими разделами:  
формулировка задания; 
    - гиперссылка на каталог репозитория с исходными текстами, наборами данных и др. сопутствующими материалами; 
    - рисунки с результатами визуализации; 
    - пояснения, раскрывающие смысл полученных результатов.

In [39]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [40]:
train = pd.read_csv('train.csv', header=None)
test = pd.read_csv('test.csv', header=None)

In [41]:
column_names = {
    0: "age",
    1: "workclass",
    2: "fnlwgt",
    3: "education",
    4: "education-num",
    5: "marital-status",
    6: "occupation",
    7: "relationship",
    8: "race",
    9: "sex",
    10: "capital-gain",
    11: "capital-loss",
    12: "hours-per-week",
    13: "native-country",
    14: "income"
}

train.rename(columns=column_names, inplace=True)
test.rename(columns=column_names, inplace=True)
test['income'] = test['income'].str.rstrip('.')

In [42]:
def classification_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=' >50K')
    recall = recall_score(y_true, y_pred, pos_label=' >50K')
    f1 = f1_score(y_true, y_pred, pos_label=' >50K')
    return accuracy, precision, recall, f1

def classify_dataset(dataset, criterion, test_size=0.2):
    # Преобразование категориальных признаков в числовые с помощью One-Hot Encoding
    dataset_encoded = pd.get_dummies(dataset, columns=dataset.drop(columns=['income']).columns)

    # Разделение на признаки и метки
    X = dataset_encoded.drop(columns=['income'])
    y = dataset_encoded['income']

    # Разделение на обучающий и тестовый наборы
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Создание и обучение модели дерева решений
    model = DecisionTreeClassifier(criterion=criterion, random_state=42)
    model.fit(X_train, y_train)

    # Предсказание на тестовом наборе
    y_pred = model.predict(X_test)

    # Вычисление показателей качества классификации
    accuracy, precision, recall, f1 = classification_metrics(y_test, y_pred)

    return accuracy, precision, recall, f1

In [43]:
dataset = pd.concat([train, test])
dataset

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [52]:
# 
print("Количество записей с значением  <=50K  --> ", len(dataset[dataset['income'] == ' <=50K']))
print("Количество записей с значением  >50K  --> ", len(dataset[dataset['income'] == ' >50K']))

Количество записей с значением  <=50K  -->  37155
Количество записей с значением  >50K  -->  11687


In [47]:
# Классификация gini, entropy
criterion = 'gini'
accuracy, precision, recall, f1 = classify_dataset(dataset, criterion, 0.2)

print(f"Метрики с критерием {criterion}:\n" +
        f"Accuracy оценка: {accuracy:.2f}\n" +
        f"Precision оценка: {precision:.2f}\n" +
        f"Recall оценка: {recall:.2f}\n" +
        f"F1 оценка: {f1:.2f}"
      )

Метрики с критерием gini:
Accuracy оценка: 0.85
Precision оценка: 0.72
Recall оценка: 0.64
F1 оценка: 0.67
